In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import layers, models

# Function to extract frames from a video file
def extract_frames(video_path, num_frames=16, frame_size=(256, 256)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Function to load the dataset and labels
def load_dataset(fake_videos, real_videos, num_frames=16):
    fake_frames = [extract_frames(video) for video in fake_videos]
    real_frames = [extract_frames(video) for video in real_videos]

    fake_labels = np.zeros(len(fake_frames))
    real_labels = np.ones(len(real_frames))

    frames = np.concatenate([fake_frames, real_frames])
    labels = np.concatenate([fake_labels, real_labels])

    frames, labels = shuffle(frames, labels, random_state=42)

    return frames, labels

# Function to build the MesoNet model
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to train the model
def train_model(model, frames, labels, epochs=10, batch_size=32):
    frames = frames / 255.0  # Normalize pixel values
    frames = frames.reshape(frames.shape[0], frames.shape[1], frames.shape[2], frames.shape[3], 1)
    X_train, X_val, y_train, y_val = train_test_split(frames, labels, test_size=0.2, random_state=42)

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Example usage
fake_videos = ["path/to/fake/video1.mp4", "path/to/fake/video2.mp4"]
real_videos = ["path/to/real/video1.mp4", "path/to/real/video2.mp4"]

frames, labels = load_dataset(fake_videos, real_videos)
input_shape = frames.shape[1:]

model = build_model(input_shape)
train_model(model, frames, labels)
